In [ ]:
pip install wandb

In [ ]:
!wandb login

In [ ]:
import wandb
from sklearn.datasets import make_classifcation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib

In [ ]:
X, y = make_classification(
    n_samples = 1000,
    n_features = 10,
    n_classes = 2,
    weights = [0.9, 0.1],
    random_state = 42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
params = {
    'C':1.0,
    'solver':'lblinear',
    'random_state':42
}

In [ ]:
lr_model = LogisticRegression(**params)

In [ ]:
lr_model.fit(X_train, y_train)

In [ ]:
y_pred = lr_model.predict(X_test)
y_prob = lr_model.predict_proba(X_test)

In [ ]:
report=classification_report(y_test, y_pred, output_dict=True)

In [ ]:
with wandb.init(
    project='LogisticRegression_classification',
    name='lr_model',
    job_type='train'
) as run:
    run.config.update(params)

    metrics_to_log = {
        "accuracy": report['accuracy'],
        "recall_class_1": report['1']['recall'],
        "f1_class_1": report['1']['f1-score']
    }

    run.log(metrics_to_log)

    wandb.sklearn.plot_confusion_matrix(y_test, y_pred, labels=lr_model.classes_)
    wandb.sklearn.plot_roc(y_test, y_prob, labels=lr_model.classes_)

    model_filename = 'Logistic_regression_model.joblib'
    joblib.dump(lr_model, model_filename)

    artifact = wandb.Artifact(
        name='logistic_regression_model',
        type='model',
        description='A logistic regression model for binary classification'
    )
    artifact.add_file(model_filename)

    run.log_artifact(artifact)